In [1]:
import pandas as pd
# 5.2
# Сколько уникальных фильмов представлено в таблице movies?

mv_data = pd.read_csv('.\data\movies\movies.csv')
mv_data.shape[0]



9742

In [4]:
# 5.3
# Сколько уникальных пользователей в таблице ratings1?
rt1_data = pd.read_csv('./data/movies/ratings1.csv')
rt1_data.userId.nunique()

274

In [10]:
# 5.4
# В каком году было выставлено больше всего оценок?
# Для ответа на этот вопрос используйте таблицу dates.
date_data = pd.read_csv('./data/movies/dates.csv')
date_data.date = pd.to_datetime(date_data.date)
date_data.groupby(date_data.date.dt.year).count().idxmax()

date    2000
dtype: int64

In [24]:
# объединяем таблицы рейтингов
rt2_data = pd.read_csv('./data/movies/ratings2.csv')
ratings = pd.concat([rt1_data, rt2_data], ignore_index=True)

# удаляем дубликаты и перезаписываем индексы
ratings.drop_duplicates(ignore_index=True, inplace=True)

In [26]:
rating_dates = pd.concat([ratings, date_data], axis=1)

In [29]:
full_data = rating_dates.merge(mv_data, on='movieId', how='left')
#mv_data.columns

In [32]:
full_data.date

0        2000-07-30 18:45:03
1        2000-07-30 18:20:47
2        2000-07-30 18:37:04
3        2000-07-30 19:03:35
4        2000-07-30 18:48:51
                 ...        
100831   2017-05-03 21:53:22
100832   2017-05-03 22:21:31
100833   2017-05-08 19:50:47
100834   2017-05-03 21:19:12
100835   2017-05-03 21:20:15
Name: date, Length: 100836, dtype: datetime64[ns]

In [115]:
# 8.1
# Создайте в таблице новый признак year_release, который соответствует году 
# выпуска фильма.
import re
# У скольких фильмов не указан год их выпуска?
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

full_data['year release'] = full_data.title.apply(get_year_release)
full_data['year release'].isna().sum()

18

In [61]:
# 8.2
# Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку 
# зрителей?
# В качестве ответа запишите название этого фильма без указания года его выпуска.
full_data.groupby(['year release', 'title']).mean().loc[1999].rating.idxmin()

'Bloodsport: The Dark Kumite (1999)'

In [67]:
# 8.3
# Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
# Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).
full_data.groupby(['year release', 'genres']).mean().loc[2010].rating.idxmin()

'Action|Sci-Fi'

In [79]:
# 8.4
# Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) 
# комбинаций жанров (genres) фильмов? В качестве ответа запишите идентификатор этого пользователя.
full_data.groupby(['userId']).nunique()['genres'].idxmax()

599

In [83]:
#  8.5
# Найдите пользователя, который выставил наименьшее количество оценок, 
# но его средняя оценка фильмам наибольшая.
# В качестве ответа укажите идентификатор этого пользователя.
# Чтобы рассчитать несколько параметров для каждого пользователя 
# (количество оценок и среднюю оценку), можно воспользоваться методом agg() 
# на сгруппированных данных.
full_data.groupby(['userId'])['rating'].agg(['count', 'mean']) \
    .sort_values(by=['count', 'mean'], ascending=[True, False]) \
    .index[0] 

53

In [90]:
# 8.6
# Найдите сочетание жанров (genres) за 2018 году, которое имеет наибольший 
# средний рейтинг (среднее по столбцу rating), и при этом число выставленных 
# ему оценок (количество значений в столбце rating) больше 10.
# Запишите сочетание так же, как оно указано в таблице 
# (через разделитель |, без пробелов).
# mask_year = full_data[]
full_data[full_data['year release'] == 2018].groupby('genres')['rating'] \
    .agg(['mean', 'count']).query('count > 10')['mean'].idxmax()

'Action|Adventure|Sci-Fi'

In [93]:
# 8.7
# Добавьте в таблицу новый признак year_rating — год выставления оценки. 
# Создайте сводную таблицу, которая иллюстрирует зависимость среднего рейтинга 
# фильма от года выставления оценки и жанра. Выберите верные варианты ответа, 
# исходя из построенной таблицы:

def get_year_rating(time):
    if time:
        return time.year
    return None
full_data['year_rating'] = full_data['date'].apply(get_year_rating)
pv = full_data.pivot_table(index='year_rating', values='rating', columns='genres', aggfunc='mean')

In [96]:
# A. За весь период (с 1996 по 2018 год) сочетание жанров Action|Adventure 
#    ни разу не получало среднюю оценку ниже 3.
pv['Action|Adventure'].min() >= 3

True

In [99]:
# B. Наилучшую оценку жанр Action|Adventure|Animation|Children|Comedy|IMAX 
#    получил в 2010 году.
pv['Action|Adventure|Animation|Children|Comedy|IMAX'].idxmax() == 2010

False

In [107]:
# C. Среди сочетаний жанров, получивших наивысшую среднюю оценку в 2018 году, 
#    есть сочетание Animation|Children|Mystery.
pv['Animation|Children|Mystery'].loc[2018] == pv.loc[2018].max()

True

In [114]:
# D. Для жанра Comedy прослеживается тенденция падения рейтинга с каждым годом 
#    (с 1996 по 2018).
all(pv['Comedy'].sort_values(ascending=False).index == range(1996, 2019))

False